#### ID 2040

```Summarize the number of customers and transactions for each month in 2017, keeping transactions that were greater or equal to $5.```

In [ ]:
%%sql
WITH cte AS (SELECT transaction_id, SUM(sales)
             FROM wfm_transactions
             WHERE EXTRACT(YEAR FROM transaction_date) = 2017
             GROUP BY transaction_id
             HAVING SUM(sales) >= 5)
SELECT EXTRACT(MONTH FROM transaction_date) AS month,
       COUNT(DISTINCT customer_id)          AS customers,
       COUNT(DISTINCT transaction_id)       AS transactions
FROM cte
         JOIN wfm_transactions USING (transaction_id)
GROUP BY month
ORDER BY month

In [ ]:
df = wfm_transactions
df['year'] = df['transaction_date'].dt.year
df['month'] = df['transaction_date'].dt.month
transactions_list = df.query('year == 2017').groupby('transaction_id', as_index=False).agg(total_sales=('sales', 'sum')).query('total_sales >= 5')['transaction_id'].to_list()
df.query('transaction_id.isin(@transactions_list)').groupby('month', as_index=False).agg(customers=('customer_id', 'nunique'), transactions=('transaction_id', 'nunique'))